In [223]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [241]:
# espn site text
t1 = requests.get('http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9')
t2 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=50")
t3 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=100")
t4 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=150")
t5 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=200")
t6 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=250")
t7 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=300")
t8 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=350")
t9 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=400")

In [242]:
ranks = [t1,t2,t3,t4,t5,t6,t7,t8,t9]
pt=[]
for r in ranks:
    pt.append(r.text)

In [243]:
rank_tbls=[]
for text in pt:
    bs=BeautifulSoup(text,"lxml")
    table = bs.find(lambda tag: tag.name=='table' and tag.has_key('id') and tag['id']=="playertable_0")
    df = pd.read_html(str(table))
    ranktable = df[0].iloc[1:len(df[0]),[0,1,3,6,7,8,9,10,11,12,13,15]]
    colnames = ranktable.iloc[0,0:(len(ranktable.columns)-1)]
    ranktable.columns = list(colnames)+["2018 PlayerRating"]
    rankings = ranktable.iloc[1:len(ranktable),:].set_index("RNK")
    rankings["position"] = rankings["PLAYER, TEAM POS"].apply(lambda p: p[-2:])
    rankings["PLAYER, TEAM POS"] = rankings["PLAYER, TEAM POS"].apply(lambda p: p.split(",")[0])
    rank_tbls.append(rankings)
    

In [244]:
rankings = pd.concat(rank_tbls)

In [245]:
league = ['WONG', 'Firo', 'PHAN', 'JETH', 'DHIN', 'MONE', 'HDP', 'cc', 'SYED', 'BOBK']

In [246]:
team_players={}
buckets=[]
for owner in league:
    team = rankings.loc[rankings["TYPE"]==owner]
    teamtop50 = len(rank_tbls[0].loc[rankings["TYPE"]==owner])
    teamtop100 = len(rank_tbls[1].loc[rankings["TYPE"]==owner])+teamtop50
    teamtop150 = len(rank_tbls[2].loc[rankings["TYPE"]==owner])+teamtop100
    teamtop200 = len(rank_tbls[3].loc[rankings["TYPE"]==owner])+teamtop150
    
    team_players[owner] = [sum(team["2018 PlayerRating"]), teamtop50,
                           teamtop100,teamtop150,teamtop200]

In [247]:
teamperformance = pd.DataFrame(pd.Series(team_players))
teamperformance["TeamRating"] = teamperformance[0].apply(lambda s: s[0])
teamperformance["nTop50Players"] = teamperformance[0].apply(lambda s: s[1])
teamperformance["nTop100Players"] = teamperformance[0].apply(lambda s: s[2])
teamperformance["nTop150Players"] = teamperformance[0].apply(lambda s: s[3])
teamperformance["nTop200Players"] = teamperformance[0].apply(lambda s: s[4])
standings = teamperformance.iloc[:,1:len(teamperformance.columns)].sort("TeamRating",ascending=False)

/Users/eugenewong/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [248]:
standings

,TeamRating,nTop50Players,nTop100Players,nTop150Players,nTop200Players
WONG,101.08,6,13,13,13
Firo,94.39,6,12,13,13
HDP,90.55,4,11,13,13
PHAN,85.66,5,10,12,13
SYED,74.89,6,9,10,10
JETH,74.11,5,7,10,12
BOBK,72.44,5,9,13,13
MONE,70.94,6,9,9,10
cc,58.04,5,8,9,11
DHIN,41.79,2,5,7,10


In [249]:
rankings.loc[rankings["TYPE"]=="WONG"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
2,LeBron James,WONG,4.65,0.33,1.58,1.70,4.09,1.48,2.00,3.40,19.24,PF
21,Joel Embiid,WONG,1.49,-0.26,-0.03,2.41,0.75,0.29,3.12,1.88,9.66,C
30,Al Horford,WONG,1.62,0.29,0.74,2.07,1.71,0.29,0.87,0.95,8.54,C
33,Jayson Tatum,WONG,0.86,1.46,0.95,1.28,0.05,1.12,1.44,1.26,8.42,SF
36,DeMar DeRozan,WONG,0.95,1.28,0.25,0.53,1.71,1.36,-0.39,2.59,8.29,SG
42,Tyreke Evans,WONG,1.25,0.68,1.23,0.79,1.14,0.89,0.03,1.57,7.57,SG
51,Jrue Holiday,WONG,0.18,0.64,0.25,0.68,2.64,1.24,-0.25,1.18,6.56,SG
61,Lou Williams,WONG,-0.20,1.71,1.58,-0.17,1.28,0.65,-0.53,1.48,5.79,SG
62,Eric Gordon,WONG,-1.42,1.84,3.05,-0.37,0.65,0.17,0.03,1.84,5.79,TD


In [250]:
rankings.loc[rankings["TYPE"]=="Firo"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
6,Kevin Durant,Firo,2.26,1.78,2.00,1.21,1.61,0.29,3.83,2.36,15.34,TD
11,Victor Oladipo,Firo,0.11,0.83,2.07,0.89,1.38,2.43,1.44,2.52,11.66,SG
15,Draymond Green,Firo,0.20,0.19,0.53,1.73,3.07,1.48,2.84,0.48,10.52,TD
45,Aaron Gordon,Firo,1.68,-0.60,1.51,1.66,0.25,0.53,0.73,1.43,7.19,SF
46,Ricky Rubio,Firo,-1.78,2.61,0.60,0.32,2.07,2.79,-0.39,0.96,7.17,PG
48,Joe Ingles,Firo,-0.07,0.56,2.42,0.64,1.31,1.72,-0.25,0.59,6.92,SF
55,Gary Harris,Firo,1.37,-0.28,1.86,-0.22,0.48,2.19,0.03,0.87,6.3,SG
58,Dewayne Dedmon,Firo,2.81,-0.54,-0.03,2.03,-0.15,-0.18,1.44,0.62,6,C
59,Kent Bazemore,Firo,-1.37,0.35,0.88,0.53,1.18,2.79,0.59,1.01,5.95,SG


In [251]:
rankings.loc[rankings["TYPE"]=="HDP"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
5,Anthony Davis,HDP,3.38,0.86,0.11,2.97,0.78,1.60,3.69,2.77,16.15,C
9,Damian Lillard,HDP,-1.85,3.97,2.35,0.87,2.70,1.36,0.59,2.80,12.79,PG
12,Kristaps Porzingis,HDP,0.42,1.79,1.30,1.41,-0.28,-0.18,4.39,2.72,11.57,PF
25,Nikola Vucevic,HDP,1.14,0.30,1.37,1.90,0.85,0.89,1.44,1.57,9.45,C
52,Jeremy Lamb,HDP,0.37,1.23,1.02,0.79,0.91,0.53,0.17,1.36,6.37,SG
54,Goran Dragic,HDP,0.25,0.94,1.16,0.44,1.74,0.65,-0.53,1.71,6.36,PG
60,Reggie Jackson,HDP,-0.10,1.73,0.81,0.02,2.50,0.53,-0.82,1.18,5.86,PG
84,Avery Bradley,HDP,-0.16,0.31,1.65,0.02,0.15,1.72,-0.39,1.46,4.75,SG
91,Lauri Markkanen,HDP,-0.96,0.27,1.79,1.73,-0.08,-0.06,0.45,0.95,4.08,PF


In [252]:
rankings.loc[rankings["TYPE"]=="SYED"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
10,Nikola Jokic,SYED,1.19,1.48,0.88,3.16,1.71,1.96,0.87,1.37,12.62,C
16,Bradley Beal,SYED,0.56,1.15,2.00,0.74,1.08,1.48,0.31,2.70,10.02,SG
19,Devin Booker,SYED,-0.42,1.93,2.35,0.62,1.87,0.77,-0.11,2.73,9.73,SG
23,Evan Fournier,SYED,1.08,1.94,2.07,0.23,1.01,1.36,-0.11,1.90,9.48,SF
37,Andre Drummond,SYED,1.46,-2.68,-0.86,4.44,1.04,1.84,2.00,0.92,8.14,C
44,Blake Griffin,SYED,-1.24,0.08,1.65,1.77,1.94,0.77,0.03,2.31,7.31,PF
53,DeAndre Jordan,SYED,3.08,-1.80,-0.86,3.91,-0.35,-0.06,2.00,0.46,6.37,C
90,Julius Randle,SYED,1.91,-1.38,-0.58,1.70,0.22,-0.06,1.44,0.93,4.16,PF
92,Malcolm Brogdon,SYED,0.06,0.64,1.09,-0.22,1.04,1.00,-0.39,0.75,3.97,PG


In [253]:
rankings.loc[rankings["TYPE"]=="PHAN"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
3,DeMarcus Cousins,PHAN,0.23,0.38,2.00,3.91,2.34,2.43,2.98,3.07,17.33,PF
8,Karl-Anthony Towns,PHAN,2.49,1.51,0.81,3.31,-0.02,0.05,2.84,2.13,13.13,C
17,Marc Gasol,PHAN,-0.92,1.05,1.09,2.32,1.21,-0.30,3.55,1.78,9.79,C
34,Kyle Lowry,PHAN,-0.47,1.63,2.14,0.96,3.07,0.29,-0.39,1.17,8.39,PG
50,Dennis Schroder,PHAN,-0.31,1.54,0.60,-0.15,2.87,1.24,-0.82,1.66,6.63,PG
57,Tim Hardaway Jr.,PHAN,-0.92,0.65,2.07,0.53,0.98,1.36,-0.25,1.64,6.07,SF
76,Donovan Mitchell,PHAN,-2.51,0.99,1.51,0.19,1.08,1.84,0.73,1.29,5.11,PG
86,Rodney Hood,PHAN,-1.20,1.83,2.49,-0.15,0.12,0.17,-0.25,1.51,4.51,SG
94,Ryan Anderson,PHAN,-0.24,0.60,2.49,0.96,-0.51,-0.42,0.31,0.70,3.89,TD


In [238]:
rankings.loc[rankings["TYPE"]=="MONE"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
5,Giannis Antetokounmpo,MONE,3.47,-0.12,-0.27,2.59,1.56,2.00,3.57,3.28,16.08,TD
12,Kyrie Irving,MONE,0.45,2.23,1.99,0.12,2.26,2.37,0.07,2.42,11.92,PG
26,LaMarcus Aldridge,MONE,1.45,0.73,-0.20,2.14,0.48,0.24,2.26,2.39,9.49,PF
27,CJ McCollum,MONE,0.65,1.42,2.79,0.39,0.52,1.37,-0.22,2.23,9.15,SG
32,Otto Porter Jr.,MONE,2.18,-0.19,1.41,1.51,-0.00,2.37,0.22,1.23,8.72,SF
38,Darren Collison,MONE,0.36,1.46,0.60,0.03,3.41,1.37,-0.51,0.99,7.72,PG
47,Tobias Harris,MONE,0.17,1.24,2.72,0.84,0.17,0.12,-0.22,1.83,6.87,SF
68,Paul Millsap*,MONE,-0.21,-1.20,0.60,1.11,0.87,1.00,2.11,1.16,5.44,O
72,TJ Warren,MONE,1.73,-0.84,-0.71,1.29,-0.04,0.50,1.38,1.99,5.3,SF


In [239]:
rankings.loc[rankings["TYPE"]=="DHIN"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
4,Stephen Curry,DHIN,0.39,3.82,3.30,0.75,2.75,2.62,-0.22,2.68,16.1,PG
18,Robert Covington,DHIN,0.68,0.59,3.30,0.91,-0.04,2.25,1.09,1.34,10.12,SF
62,Rudy Gobert*,DHIN,2.27,-0.76,-0.86,1.71,-0.35,-0.26,3.57,0.44,5.77,O
64,Carmelo Anthony,DHIN,-0.94,0.55,1.62,1.02,-0.04,0.62,1.09,1.67,5.59,PF
95,Trevor Ariza,DHIN,-1.15,0.85,1.62,0.57,0.48,1.62,-0.51,0.37,3.85,SF
127,Dwyane Wade,DHIN,-0.95,0.47,-0.64,0.37,1.32,0.62,1.09,0.33,2.62,SG
129,Serge Ibaka,DHIN,1.04,-0.77,0.97,0.66,-0.63,-0.63,1.38,0.49,2.51,PF
144,Wilson Chandler,DHIN,-0.18,0.40,0.24,0.50,0.62,-0.13,0.07,0.30,1.83,SF
149,Marcus Smart,DHIN,-3.94,0.22,0.60,0.48,2.09,1.87,0.07,0.27,1.67,SG


In [240]:
rankings.loc[rankings["TYPE"]=="JETH"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
1,James Harden,JETH,-0.17,2.82,4.69,0.70,4.98,2.50,0.36,3.86,19.75,PG
14,Paul George,JETH,-0.52,0.41,3.08,1.00,0.66,4.00,0.07,2.13,10.82,SF
36,Jeff Teague,JETH,-0.99,0.34,1.04,0.08,3.45,2.62,0.36,1.02,7.93,PG
39,Enes Kanter,JETH,3.60,1.18,-0.86,2.68,-0.25,0.12,0.22,0.93,7.62,C
52,Jimmy Butler,JETH,-0.84,1.66,0.16,0.79,1.39,2.50,-0.37,1.16,6.46,SF
58,Harrison Barnes,JETH,-0.25,1.10,0.97,1.71,0.14,0.50,-0.37,2.12,5.92,PF
76,Kyle Kuzma,JETH,1.38,-0.11,1.11,1.53,0.14,-0.26,-0.51,1.69,4.97,PF
109,J.J. Barea,JETH,-0.72,0.16,1.77,0.10,1.95,-0.26,-0.66,0.98,3.32,PG
110,JJ Redick,JETH,-0.36,1.24,1.48,-0.19,1.04,-0.01,-0.80,0.90,3.31,SG


In [221]:
rankings.loc[rankings["TYPE"]=="BOBK"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
17,Klay Thompson,BOBK,1.28,0.40,3.39,0.65,0.72,0.31,1.17,2.19,10.11,SG
22,Kevin Love,BOBK,-0.16,2.95,1.27,2.75,0.18,0.57,0.41,1.72,9.69,C
33,Ben Simmons,BOBK,1.77,-3.15,-0.86,2.11,3.47,2.55,0.87,1.52,8.29,PG
39,John Wall,BOBK,-0.71,-0.55,0.51,0.11,3.83,0.97,1.78,1.59,7.53,PG
46,Pau Gasol,BOBK,0.65,0.31,0.20,1.97,1.33,-0.61,2.39,0.55,6.79,C
57,Kent Bazemore,BOBK,-1.12,0.36,0.66,0.46,1.33,2.95,0.41,1.03,6.09,SG
69,Taj Gibson,BOBK,1.25,0.87,-0.55,1.83,-0.11,0.97,0.56,0.46,5.28,PF
76,Derrick Favors,BOBK,1.56,-0.17,-0.55,1.31,-0.00,0.57,1.48,0.86,5.04,C
91,Lonzo Ball,BOBK,-3.51,-2.36,0.51,1.74,3.47,2.16,1.63,0.36,3.99,PG


In [222]:
rankings.loc[rankings["TYPE"]=="cc"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
13,Clint Capela,cc,4.36,-1.64,-0.86,3.48,-0.11,0.71,3.91,1.08,10.92,C
25,Kemba Walker,cc,0.08,2.60,1.95,0.15,2.64,0.18,-0.35,2.30,9.55,PG
34,Brook Lopez,cc,0.30,0.83,1.04,0.82,0.32,-0.22,3.76,1.42,8.26,C
36,Russell Westbrook,cc,-1.72,-1.39,0.96,1.78,4.30,2.55,-0.50,1.78,7.78,PG
42,Khris Middleton,cc,-0.31,1.46,1.04,0.82,1.73,1.23,-0.65,1.64,6.96,SG
62,James Johnson,cc,0.75,-0.28,0.43,0.79,1.44,1.10,0.72,0.67,5.62,PF
87,Marcin Gortat,cc,1.81,-0.54,-0.86,2.23,0.03,-0.22,1.17,0.54,4.17,C
99,Wesley Matthews,cc,-1.93,0.36,2.25,0.18,1.01,1.50,-0.50,0.81,3.66,SG
136,Dirk Nowitzki,cc,-0.56,0.51,0.66,1.08,0.21,-0.22,-0.04,0.59,2.22,PF


In [202]:
rankings.loc[rankings["TYPE"]=="FA"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
81,Danny Green,FA,-0.45,-0.40,1.40,0.21,0.24,1.43,1.53,0.53,4.48,SG
85,Kyle Anderson,FA,0.69,-0.34,-0.70,1.30,0.69,0.62,1.68,0.21,4.14,PF
89,David West,FA,2.49,-0.15,-0.62,0.04,-0.09,0.21,2.15,-0.06,3.95,PF
90,Bojan Bogdanovic,FA,1.10,0.51,1.71,0.16,-0.02,0.07,-0.65,1.04,3.93,SF
92,John Collins,FA,1.66,-0.02,-0.86,1.68,-0.46,-0.33,1.53,0.65,3.85,C
93,DeMarre Carroll,FA,-0.62,-0.01,1.16,1.10,-0.13,1.43,0.13,0.76,3.82,SF
96,Marco Belinelli,FA,-0.99,1.09,1.94,-0.27,0.54,1.43,-0.80,0.73,3.66,SG
97,E'Twaun Moore,FA,1.17,-0.56,0.85,-0.08,0.54,1.29,-0.34,0.77,3.66,SG
98,Kyle Korver,FA,0.53,0.72,2.33,-0.20,-0.58,-0.06,0.13,0.61,3.47,SF


In [203]:
rankings

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
1,James Harden,JETH,-0.19,2.90,5.05,0.84,5.37,2.78,0.44,4.20,21.39,PG
2,LeBron James,WONG,4.66,0.24,1.32,1.88,4.21,1.56,2.15,3.46,19.49,PF
3,DeMarcus Cousins,PHAN,0.45,0.14,2.25,3.98,2.47,2.51,2.92,3.18,17.9,PF
4,Kevin Durant,Firo,2.44,1.78,2.09,1.42,1.84,0.21,4.32,2.59,16.68,TD
5,Giannis Antetokounmpo,MONE,3.86,-0.18,-0.24,2.67,1.58,2.24,3.39,3.35,16.67,PF
6,Anthony Davis,HDP,3.31,0.34,0.23,2.87,0.72,1.83,3.86,2.61,15.77,TD
7,Stephen Curry,DHIN,0.02,3.90,3.26,0.62,2.73,2.51,-0.18,2.56,15.42,PG
8,Damian Lillard,HDP,-1.76,3.92,2.17,0.98,2.91,1.56,0.60,2.78,13.18,PG
9,Karl-Anthony Towns,PHAN,2.46,1.36,0.70,3.06,-0.17,0.21,3.08,2.01,12.7,C


In [25]:
testdf=pd.DataFrame({"bin":[0,1,2,3,4,5,6,7],
             "hits":[50,1,5,2,0,4,2,0]})

In [26]:
testdf["rolling"] = testdf["bin"].cumsum()

In [28]:
def mapper(row):
    row["hits"]/2343*50

In [43]:
testmap = pd.read_csv("testmapper.csv")

In [44]:
testmap.head()


,Bins,Contribution,accum
0,-10.00,18,99.456398
1,-9.99,18,99.455602
2,-9.98,0,99.454806
3,-9.97,0,99.454806
4,-9.96,0,99.454806


In [45]:
upper = 1130837
lower = 3437820
midpt=50

In [110]:
pos = testmap.loc[testmap["Bins"]>=0]
neg = testmap.loc[testmap["Bins"]<0]

In [115]:
neg = neg[::-1].reset_index(drop=True)

In [80]:
pos["step1"] = pos["Contribution"]/upper*midpt

/Users/eugenewong/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [99]:
pos=pos.reset_index(drop=True)

In [83]:
sum(pos["Contribution"])

3451719

In [116]:
# mapped=[50]
# for i in range(len(pos["Contribution"])-1):
#     mapped.append(mapped[i]+(float(pos["Contribution"][i])/sum(pos["Contribution"]))*50)
    
mapped=[50]
for i in range(len(neg["Contribution"])-1):
    mapped.append(mapped[i]-(float(neg["Contribution"][i])/sum(neg["Contribution"]))*50)

In [117]:
sum(neg["Contribution"])

1118542

In [118]:
neg["mapped"] = mapped

In [119]:
neg

,Bins,Contribution,accum,step1,step2,mapped
0,-0.01,24589,51.087204,1.087204,49.456376,50.000000
1,-0.02,28242,52.335926,1.248721,48.369173,48.900846
2,-0.03,25011,53.441788,1.105862,47.120451,47.638399
3,-0.04,24998,54.547076,1.105287,46.014589,46.520381
4,-0.05,27213,55.750301,1.203224,44.909302,45.402944
5,-0.06,22606,56.749826,0.999525,43.706078,44.186495
6,-0.07,23893,57.806257,1.056430,42.706553,43.175983
7,-0.08,22076,58.782348,0.976091,41.650123,42.107941
8,-0.09,22144,59.761446,0.979098,40.674032,41.121120
9,-0.10,22289,60.746956,0.985509,39.694934,40.131260


In [92]:
pos.to_csv("distmap.csv")

In [60]:
pos["step2"] = pos["step1"].cumsum()

/Users/eugenewong/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
pos

,Bins,Contribution,accum,step1,step2
1002,0.01,27711,45.100194,1.225243,1.225243
1003,0.02,29311,44.673891,1.295987,2.521230
1004,0.03,28450,44.260112,1.257918,3.779148
1005,0.04,32879,43.781916,1.453746,5.232894
1006,0.05,29147,43.357999,1.288736,6.521630
1007,0.06,29648,42.926796,1.310887,7.832517
1008,0.07,30844,42.478197,1.363769,9.196286
1009,0.08,29600,42.047692,1.308765,10.505051
1010,0.09,29467,41.619121,1.302885,11.807935
1011,0.10,28311,41.207363,1.251772,13.059707
